In [4]:
!pip install -q simpletransformers gdown

In [26]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import gdown
import zipfile
import os

# Load Model from ggdrive

In [68]:
def download_model(extract_path = "models/"):
    file_id = '1WYkuOxW9wg442Gbsw93vh3Vyh9qoWb4r'
    destination = 'model.zip'
    url = f'https://drive.google.com/uc?id={file_id}'

    gdown.download(url, destination, quiet=False)
    with zipfile.ZipFile(destination, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    extracted_files = os.listdir(extract_path)
    print(extracted_files)
    os.remove(destination)
    return extract_path

In [69]:
extract_path = "../models/"
save_path = download_model(extract_path)
save_path = os.path.join(save_path, "model_zip")
best_model = ClassificationModel(
    "distilbert",
    save_path,
    use_cuda=False
)

Downloading...
From (original): https://drive.google.com/uc?id=1WYkuOxW9wg442Gbsw93vh3Vyh9qoWb4r
From (redirected): https://drive.google.com/uc?id=1WYkuOxW9wg442Gbsw93vh3Vyh9qoWb4r&confirm=t&uuid=cb6cf169-9db2-43ac-a486-0fef0eaf342c
To: /Users/metissotangkur/Desktop/advanced_computing/nlp/coursework/dontpatronizeme/code/model.zip
100%|██████████| 247M/247M [00:08<00:00, 30.3MB/s] 


['.DS_Store', 'model_zip', '__MACOSX', 'model_zip.zip']


In [66]:
def save_line_pred(pred_list, file_name):
    with open(file_name, 'w') as file:
        for val in pred_list:
            file.write(f"{val}\n")

# Predict DEV

In [51]:
dev_path = "../data/dev_df.csv"

cols = ["text", "target_flag"]

dev_df = pd.read_csv(dev_path, sep="\t")

dev_df.head()

,Unnamed: 0,par_id,keyword,country_code,text,target_flag,weights
0,0,107,homeless,ke,"His present "" chambers "" may be quite humble ,...",1.0,1
1,1,149,disabled,us,Krueger recently harnessed that creativity to ...,1.0,1
2,2,151,poor-families,in,10:41am - Parents of children who died must ge...,1.0,1
3,3,154,disabled,ng,When some people feel causing problem for some...,1.0,1
4,4,157,poor-families,ca,We are alarmed to learn of your recently circu...,1.0,1


In [52]:
dev_df.loc[dev_df["text"].isna(), "text"] = ""

In [70]:
dev_result, dev_model_outputs, dev_wrong_predictions = best_model.eval_model(dev_df[cols])

/Users/metissotangkur/anaconda3/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

In [71]:
dev_result

{'mcc': 0.4717101152985508,
 'accuracy': 0.9097421203438395,
 'f1_score': 0.5215189873417722,
 'tp': 103,
 'tn': 1802,
 'fp': 93,
 'fn': 96,
 'auroc': 0.898357221463518,
 'auprc': 0.5599527964727997,
 'eval_loss': 0.21991355468829474}

In [55]:
dev_text_list = dev_df['text'].tolist()

dev_pred, dev_raw_outputs = best_model.predict(dev_text_list)

  0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/21 [00:00<?, ?it/s]

In [56]:
save_line_pred(dev_pred, "../dev.txt")

# Predict Test

In [57]:
url = "https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv"

test_df = pd.read_csv(url, sep='\t', names=["idx", "_id", "keyword", "country_code", "text"])
test_df.head()

,idx,_id,keyword,country_code,text
0,t_0,@@7258997,vulnerable,us,"In the meantime , conservatives are working to..."
1,t_1,@@16397324,women,pk,In most poor households with no education chil...
2,t_2,@@16257812,migrant,ca,The real question is not whether immigration i...
3,t_3,@@3509652,migrant,gb,"In total , the country 's immigrant population..."
4,t_4,@@477506,vulnerable,ca,"Members of the church , which is part of Ken C..."


In [58]:
test_pred, test_raw_outputs = best_model.predict(test_df["text"].tolist())

  0%|          | 0/7 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/39 [00:00<?, ?it/s]

In [59]:
save_line_pred(test_pred, "../test.txt")